In [2]:
import pandas as pd
import numpy as np
import joblib


# Метод кластериции (для демонстрации используем ввиде функции)
def clusterization(df):

  # Нормализация
  def normalize(df):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler() # (xi - min)/(max - min)
    columns = df.columns
    x = df.values
    x_scaled = scaler.fit_transform(x)
    df = pd.DataFrame(x, columns=columns)
    joblib.dump(scaler, 'models/min_max_scaler.pkl')
    return df

  # Нормализуем датасет и сохраняем модель
  normalize(df)

  # Кластеризация
  from sklearn.cluster import KMeans
  model = KMeans(random_state=1) # create model (without parameters)

  # Ищем оптимальную разбивку на кластеры
  from yellowbrick.cluster import KElbowVisualizer, InterclusterDistance
  visualizer = KElbowVisualizer (model, k=(2, 10), metrics='silhouette') # Графически ищем лучшую кластеризация диапазон от 2 до 10, Метрика: Силуэт
  visualizer.fit(df)
  visualizer.show()

  # Кластеризация лучшая модель (для примера лучшая модель 2 кластера на графике visualizer)
  model = KMeans(random_state=1, n_clusters=2)
  visualizer = InterclusterDistance(model)
  visualizer.fit(df)
  visualizer.show() # смотрим межкластерное расстояние

  cluster_labels = visualizer.fit_predict(df) # обучаем и предсказываем

  # Разделяем на train и test
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(df, cluster_labels, random_state=1)

  # 
  from sklearn.naive_bayes import MultinomialNB
  clf = MultinomialNB()
  clf.fit(X_train, y_train)
  pred = clf.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix
  print(f'Currency: {classification_report(pred, y_test)}')
  print(confusion_matrix(pred, y_test))

  joblib.dump(clf, 'models/classification.pkl')